In [1]:
#IMPORTS 
import sys
import numpy as np
import matplotlib.pyplot as plt
sys.path.append(r"C:\Users\EmilR\Documents\GitHub\MoleculeSim\ownSim")

import molecules as mc
import forces as fc
import random
import copy

In [2]:
import itertools
def grid_points(box_size,radius, spacing):
  x = np.arange(radius + spacing, box_size[0]- radius- spacing, step = 2*radius + spacing, dtype=float)
  y = np.arange(radius+ spacing, box_size[1]- radius- spacing, step = 2*radius + spacing, dtype=float)
  z = np.arange(radius+ spacing, box_size[2]- radius- spacing, step = 2*radius + spacing, dtype=float)
  all_combinations = list(itertools.product(x,y,z))
  print(x,y,z)
  return [np.array(comb) for comb in all_combinations]

In [3]:
def lin_repulsion(x):
  return max(-(1/3)*x + 10,0) 

In [4]:
# plt.figure()
# x = [float(10)]#np.arange(0,100)
# f = [f(xi) for xi in x]
# plt.plot(x,f)
# plt.show()
#print(wut(float(30)))

In [5]:
SIGMA = 0.006 #grafting density
H = 7 #width of the polyelectrolyte brush in nm

def cust_force(m1,m2):
  h = (mc.dist(m1,m2)-m1.radius-m2.radius)
  # print("h: ",h)
  # print(type(h))
  # print("f(h): ",f(h))

  return lin_repulsion(h)


  #return fc.vdw(m1,m2) #*10**(-4.6)

In [6]:
# Parameters for universe initialization
box_const_size = 850
box_size = np.array([box_const_size,box_const_size,box_const_size]) #max coordinate values for universe
#num_molecules = 5
num_steps = 100000
radius  = 100
num_molecules = 30
#other setup
seed = 1234#1313
spacing = 10
#combs = grid_points(box_size, radius, spacing)
#molecules = [mc.Molecule(pos, radius) for pos in combs]

#uni = mc.SimpleUniverse(box_size, len(molecules),np.repeat(radius,len(molecules)),seed, own_molecules = molecules, force_fun = cust_force)
uni = mc.SimpleUniverse(box_size, num_molecules,np.repeat(radius,num_molecules),seed, force_fun = cust_force)


In [7]:
box_vol = 850**3
print(box_vol/((4/3)*radius**3*np.pi))

print(30/146)

146.61154413946534
0.2054794520547945


In [8]:
print(type(uni.molecules[0].pos[0]))

<class 'numpy.float64'>


In [9]:
#calculate density:


In [10]:
#step loop
points = []

# Perform Simulation
for i in range(num_steps):
  #randomly select a molecule
  uni.make_step()
  #store intermediate results?
  points.append(uni.get_state())

acc, rej, accs, num = uni.get_stats()
print(f"Accepted: {acc}, Rejected: {rej}, Rate: {acc/num}")

Accepted: 93123, Rejected: 6877, Rate: 0.93123


In [11]:
# get only some steps
xs = np.array([inner_list[0] for sublist in points for inner_list in sublist]).reshape((num_steps,num_molecules))
ys = np.array([inner_list[1] for sublist in points for inner_list in sublist]).reshape((num_steps,num_molecules))
zs = np.array([inner_list[2] for sublist in points for inner_list in sublist]).reshape((num_steps,num_molecules))


In [12]:
xs = [xs[i] for i in range(len(xs)) if i % 100 == 0]
ys = [ys[i] for i in range(len(ys)) if i % 100 == 0]
zs = [zs[i] for i in range(len(zs)) if i % 100 == 0]

In [13]:
print(xs[0])

[640.8717017  274.4251626  160.34984787 150.67906056 274.42065334
 326.61738996 640.56051843 436.55088702 742.25869508 212.50372749
 570.83972341 584.58418404 428.14100288 217.03772024 180.47432763
 655.72697357 531.02310484 376.56803571 615.26768288 585.44097723
 160.30882791 197.92447392 400.04589505 171.89966063 595.5262658
 658.15355006 410.90035066 740.91325152 400.49223239 102.17703158]


In [14]:
#animate:
from __future__ import absolute_import, division, print_function
from mayavi import mlab
import numpy as np
import math
num_molecules = len(uni.molecules)


mlab.clf()
#mlab.points3d(0,0,0)
plt = mlab.points3d(points[0][:, 0], points[0][:, 1], points[0][:, 2], resolution = 50)
mlab.axes()
text_position = (0,0,0)
text_actor = mlab.text3d(500,150, 510, "TESTING", scale = 40, color = (0,0,0))
@mlab.animate(delay = 100, ui = True, support_movie = True) #delay = 100
def anim():
    f = mlab.gcf()
    while True:
        #temp = "step: "
        for step in range(len(xs)):#num_steps
            x = xs[step]#points[step][:, 0]
            y = ys[step]#points[step][:, 1]
            z = zs[step]#points[step][:, 2]
            plt.mlab_source.set(x=x, y=y, z=z)
            text_actor.text = f"Frame: {step}"
            
            yield


anim()
mlab.show()